<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases"/> <br>

In [22]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

print(x_train.shape)
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

baseMapNum = 32
weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))


(50000, 3, 32, 32)


In [23]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_18 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_18 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_25 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 activation_19 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_19 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                     

In [24]:
lrate = 0.01 
epochs = 18
decay = lrate/epochs 
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False) 
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [8]:
#pip install intel-tensorflow

In [14]:
#print(X_train.shape)
# import numpy as np
# X_train = np.transpose(X_train, (0, 2, 3, 1))
# print(X_train.shape)
# X_test = np.transpose(X_test, (0, 2, 3, 1))
# print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [27]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)


In [32]:
import tensorflow
import keras
batch_size = 128
epochs = 6
opt_rms = tensorflow.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75.h5')

opt_rms = tensorflow.keras.optimizers.RMSprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100.h5')

Epoch 1/18


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


390/390 [==============================] - 26s 63ms/step - loss: 0.5602 - accuracy: 0.8551 - val_loss: 0.5656 - val_accuracy: 0.8640
Epoch 2/18
390/390 [==============================] - 25s 63ms/step - loss: 0.5607 - accuracy: 0.8563 - val_loss: 0.5931 - val_accuracy: 0.8516
Epoch 3/18
390/390 [==============================] - 25s 63ms/step - loss: 0.5582 - accuracy: 0.8576 - val_loss: 0.5447 - val_accuracy: 0.8669
Epoch 4/18
390/390 [==============================] - 24s 63ms/step - loss: 0.5580 - accuracy: 0.8556 - val_loss: 0.5830 - val_accuracy: 0.8557
Epoch 5/18
390/390 [==============================] - 24s 62ms/step - loss: 0.5528 - accuracy: 0.8594 - val_loss: 0.5432 - val_accuracy: 0.8678
Epoch 6/18
390/390 [==============================] - 26s 68ms/step - loss: 0.5476 - accuracy: 0.8622 - val_loss: 0.7135 - val_accuracy: 0.8284
Epoch 7/18
390/390 [==============================] - 24s 62ms/step - loss: 0.5552 - accuracy: 0.8587 - val_loss: 0.5510 - val_accuracy: 0.8657
Epo

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  app.launch_new_instance()


390/390 [==============================] - 26s 63ms/step - loss: 0.5023 - accuracy: 0.8777 - val_loss: 0.5191 - val_accuracy: 0.8787
Epoch 2/6
390/390 [==============================] - 24s 62ms/step - loss: 0.4830 - accuracy: 0.8822 - val_loss: 0.5127 - val_accuracy: 0.8781
Epoch 3/6
390/390 [==============================] - 25s 63ms/step - loss: 0.4744 - accuracy: 0.8843 - val_loss: 0.5139 - val_accuracy: 0.8799
Epoch 4/6
390/390 [==============================] - 24s 63ms/step - loss: 0.4714 - accuracy: 0.8853 - val_loss: 0.4939 - val_accuracy: 0.8830
Epoch 5/6
390/390 [==============================] - 24s 62ms/step - loss: 0.4678 - accuracy: 0.8862 - val_loss: 0.5101 - val_accuracy: 0.8829
Epoch 6/6
390/390 [==============================] - 25s 63ms/step - loss: 0.4645 - accuracy: 0.8858 - val_loss: 0.4988 - val_accuracy: 0.8840


In [40]:
# model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=32) 
# # Final evaluation of the model 
scores = model.evaluate(x_test, y_test, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.40%


In [34]:
!pip install -qq wandb

     |████████████████████████████████| 1.8 MB 7.3 MB/s 
     |████████████████████████████████| 144 kB 64.4 MB/s 
     |████████████████████████████████| 181 kB 43.6 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


In [35]:
import wandb
print("W&B: ", wandb.__version__)
from wandb.keras import WandbCallback
wandb.login()

W&B:  0.12.16


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [39]:
wandb.init()
predictions = model(x_test)
ground_truth = y_test
wandb.log({"roc" : wandb.plot.roc_curve( ground_truth_test, predictions,labels=None, classes_to_plot=None)})